# Quantile Regression 

This file contains the implementation of all the sintetic examples that have been used in Chapter 3.

### 1. Libraries

In [ ]:
# General libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Linear Model
from sklearn.linear_model import LinearRegression, QuantileRegressor

# Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K

# Setting the style for seaborn plots
sns.set(style="white")

### 2. Linear Quantile Regression

Firstly, the sample is set.

In [ ]:
# Set seed for reproducibility
np.random.seed(123)

# Generate synthetic data
sample_normal_1 = np.random.normal(-5, 1, 1000)
sample_normal_2 = np.random.normal(5, 1, 1000)
error = np.concatenate((sample_normal_1, sample_normal_2))

x = np.random.normal(0, 1, 2000)
x_reshape = x.reshape(-1, 1)

b0 = 1
b1 = 3
y = b0 + b1*x + error
y_reshape = y.reshape(-1, 1)

The histogram of y is computed.

In [ ]:
# Plot histogram of y
plt.figure(figsize=(10, 10/1.618))
plt.hist(y, bins=100, alpha=0.8, color='skyblue', edgecolor='skyblue')
plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.show()

A linear model with least squares loss function is computed.

In [ ]:
# Fit a linear regression model
model_1 = LinearRegression()
model_1.fit(x_reshape, y)

b0_hat = model_1.intercept_
b1_hat = model_1.coef_[0]

print((b0_hat, b1_hat))

# Plot the data and the linear regression line
plt.figure(figsize=(10, 10/1.618))
plt.scatter(x, y, alpha=0.5, color="skyblue")
x_vals = np.array([x.min(), x.max()])
y_vals = b0_hat + b1_hat * x_vals
plt.plot(x_vals, y_vals, color='tomato')
plt.xlabel('x')
plt.ylabel('y')
plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.show()


Besides, a linear model with quantile regression loss function is computed. It is shown many lines, corresponding to each quantile.

In [ ]:
plt.figure(figsize=(10, 10/1.618))

plt.scatter(x, y, alpha=0.5, color="skyblue")

colormap = plt.cm.Reds
colors = [colormap(i / 8) for i in range(9)]

x_test = 1
y_test = []

div = 0.05

# Loop over quantiles and fit quantile regression models
for quantile in np.arange(div, 1, div):
    model_2 = QuantileRegressor(quantile=quantile, alpha=0)
    model_2.fit(x_reshape, y)

    b0_hat = model_2.intercept_
    b1_hat = model_2.coef_[0]

    y_test.append(b0_hat + b1_hat * x_test)

    x_vals = np.array([x.min(), x.max()])
    y_vals = b0_hat + b1_hat * x_vals

    plt.plot(x_vals, y_vals, color=colors[int(quantile / 0.1 - 1)], alpha=0.8)

plt.xlabel('x')
plt.ylabel('y')
plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.show()

Finally, the model is used with more quantiles so as to compute the density function.

In [ ]:
alturas = div/np.diff(y_test)

puntos_inicio = y_test[:-1]
puntos_fin = y_test[1:]
centros = []

plt.figure(figsize=(10, 10/1.618))

sns.kdeplot(b0+b1*1+error, fill = True,color="tomato", zorder=3)

for inicio, fin, altura in zip(puntos_inicio, puntos_fin, alturas):
    centro_x = (inicio + fin) / 2
    centros.append(centro_x)
    ancho = fin - inicio
    plt.bar(centro_x, altura, width=ancho, color='skyblue', edgecolor='skyblue', alpha=0.7)


plt.ylabel('')
plt.gca().spines['top'].set_linewidth(1.618)   
plt.gca().spines['right'].set_linewidth(1.618)  
plt.gca().spines['bottom'].set_linewidth(1.618) 
plt.gca().spines['left'].set_linewidth(1.618)  
plt.show()


### 3. Quantile Neural Network

The new sample is set.

In [ ]:
# Set seed for reproducibility
np.random.seed(123)

# Generate synthetic data
normal_sample_1 = np.random.normal(-5, 1, 1000)
normal_sample_2 = np.random.normal(5, 1, 1000)
error = np.concatenate((normal_sample_1, normal_sample_2))

x = np.random.uniform(-5, 5, 2000)
x_reshape = x.reshape(-1, 1)
x_sort = np.sort(x)
x_sort_reshape = x_sort.reshape(-1, 1)

y = x**2 + error
y_reshape = y.reshape(-1, 1)

The Y histogram is computed.

In [ ]:
plt.figure(figsize=(10, 10/1.618))

plt.hist(y, bins=100, alpha=0.8, color='skyblue', edgecolor='skyblue')

plt.gca().spines['top'].set_linewidth(1.618)   
plt.gca().spines['right'].set_linewidth(1.618)  
plt.gca().spines['bottom'].set_linewidth(1.618) 
plt.gca().spines['left'].set_linewidth(1.618)   
plt.show()

The new loss function is defined (there is no library that has the quantile loss function implemented)

In [ ]:
def quantile_loss(quantile, y_true, y_pred):
    error = y_true - y_pred
    return tf.reduce_mean(tf.maximum(quantile*error, (quantile-1)*error), axis=-1)

A neural network is computed, using quantile regression loss function with different quantiles. The result is shown, as well as the sample point.

In [ ]:
quantiles = [0.25, 0.40, 0.60, 0.75]
predictions = {}

# Fit models for each quantile and make predictions
for q in quantiles:
    model = Sequential([
        Dense(10, activation='relu', input_shape=(x_reshape.shape[1],)),
        Dense(10, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])

    model.compile(optimizer='adam', loss=lambda y_true, y_pred: quantile_loss(q, y_true, y_pred))

    model.fit(x_reshape, y_reshape, epochs=100, batch_size=32, verbose=0)
    
    predictions[q] = model.predict(np.linspace(x.min(), x.max(), 500).reshape(-1, 1))

plt.figure(figsize=(10, 6))

plt.scatter(x, y, color='skyblue', alpha=0.6)

colors = ["tomato", "r", "red", "brown"]

# Plot each quantile prediction
for i, q in enumerate(quantiles):
    plt.plot(np.linspace(x.min(), x.max(), 500), predictions[q], label=f'Quantile {q}', color=colors[i])

plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.show()

Finally, the model is used with more quantiles so as to apply it to a new observation and compute the density function.

In [ ]:
x_test = 1
y_test = []

div = 0.1
quantiles = np.concatenate(([0.001], np.arange(div, 1, div), [0.999]))

# Fit models for each quantile and make predictions for x_test
for q in quantiles:
    model = Sequential([
        Dense(10, activation='relu', input_shape=(x_reshape.shape[1],)),
        Dense(10, activation='relu'),
        Dense(10, activation='relu'),
        Dense(1, activation='linear')
    ])

    model.compile(optimizer='adam', loss=lambda y_true, y_pred: quantile_loss(q, y_true, y_pred))

    model.fit(x_reshape, y_reshape, epochs=100, batch_size=32, verbose=0)
    
    y_test.append(model.predict(np.array([x_test]).reshape(-1, 1)).flatten()[0])

heights = np.diff(quantiles)/np.diff(y_test)

start_points = y_test[:-1]
end_points = y_test[1:]

plt.figure(figsize=(10, 10/1.618))

# Plot the bars
for start, end, height in zip(start_points, end_points, heights):
    center_x = (start + end)/2
    width = end - start
    
    plt.bar(center_x, height, width=width, color='skyblue', edgecolor='skyblue', alpha=0.7)
 
plt.xlabel('Y')

# Plot the KDE
sns.kdeplot(x_test**2 + error, fill=True, color="tomato", zorder=3)

plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.ylim(bottom=0)
plt.show()

### 4. Quantile Crossing Problem

The new loss function is defined.

In [ ]:
def QuantileLoss(perc, delta=1e-4):
    perc = np.array(perc).reshape(-1)
    perc.sort()
    perc = perc.reshape(1, -1)
    def _qloss(y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I*(1 - perc) + (1 - I) * perc
        # huber loss
        huber_loss = K.sum(correction*tf.where(d <= delta, 0.5*d**2 / delta, d - 0.5*delta), -1)
        # order loss
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-2), -1)
        return huber_loss + q_order_loss
    return _qloss

As it has been done with the other neural network, firstly the model is computed with some quantiles.

In [ ]:
quantiles = [0.25, 0.40, 0.60, 0.75]

# Define and compile the model
model = Sequential([
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(quantiles))
])
model.compile(optimizer='adam', loss=QuantileLoss(quantiles))

model.fit(x_reshape, y_reshape, epochs=100, verbose=0)

predictions = model.predict(np.linspace(x.min(), x.max(), 500).reshape(-1, 1))

plt.figure(figsize=(10, 6))

plt.scatter(x, y, color='skyblue', alpha=0.6)

colors = ["tomato", "r", "red", "brown"]

for i in range(len(quantiles)):
    plt.plot(np.linspace(x.min(), x.max(), 500), predictions[:, i], label=f'Quantile {quantiles[i]}', color=colors[i])

plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.show()

Finally, the model is used to describe the density function of a new observation.

In [ ]:
div = 0.01
quantiles = np.concatenate(([0.001], np.arange(div, 1, div), [0.999]))

x_test = 1

# Define and compile the model
model = Sequential([
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(quantiles))
])
model.compile(optimizer='adam', loss=QuantileLoss(quantiles))

model.fit(x_reshape, y_reshape, epochs=100, verbose=1)

y_test = model.predict(np.array([x_test]).reshape(-1, 1)).flatten()

heights = np.diff(quantiles[::2]) / np.diff(y_test[::2])
heights = np.array([x if x > 0 else 0 for x in heights])

start_points = y_test[::2][:-1]
end_points = y_test[::2][1:]

plt.figure(figsize=(10, 10/1.618))

# Plot the bars
for start, end, height in zip(start_points, end_points, heights):
    center_x = (start + end)/2
    width = end - start
    
    plt.bar(center_x, height, width=width, color='skyblue', edgecolor='skyblue', alpha=0.7)
 
plt.xlabel('Y')

# Plot the KDE
sns.kdeplot(x_test**2 + error, fill=True, color="tomato", zorder=3)

plt.gca().spines['top'].set_linewidth(1.618)
plt.gca().spines['right'].set_linewidth(1.618)
plt.gca().spines['bottom'].set_linewidth(1.618)
plt.gca().spines['left'].set_linewidth(1.618)
plt.ylim(bottom=0)
plt.show()